In [ ]:
# | default_exp _cli

In [ ]:
# | export

import importlib
import sys
import asyncio
from pathlib import Path
from typing import *
import signal
from os import getpid
import time
import anyio
import threading
import copy
from contextlib import contextmanager
import multiprocessing
from http.server import HTTPServer, SimpleHTTPRequestHandler
import socketserver
from types import FrameType

import typer
from fastapi import FastAPI

from fastkafka.application import FastKafka
from fastkafka._components.logger import get_logger, supress_timestamps
from fastkafka._components.helpers import _import_from_string
from fastkafka.server import (
    run_fastkafka_server,
    terminate_asyncio_process,
    run_in_process,
)
from fastkafka.testing import change_dir

In [ ]:
import os
from contextlib import contextmanager
from tempfile import TemporaryDirectory

import nbformat
from nbconvert import PythonExporter
from typer.testing import CliRunner

from fastkafka.testing import change_dir
from fastkafka.server import generate_app_in_tmp

In [ ]:
# | notest

# allows async calls in notebooks

import nest_asyncio

In [ ]:
# | notest

nest_asyncio.apply()

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
runner = CliRunner()

In [ ]:
# | export

_app = typer.Typer(help="")
_run_app = typer.Typer(help="")
# _app.add_typer(_run_app, name="run")
_docs_app = typer.Typer(
    help="Collection of functions for managing fastkafka app documentation"
)
_app.add_typer(_docs_app, name="docs")


@_app.command(
    help="Runs Fast Kafka API application",
)
def run(
    num_workers: int = typer.Option(
        multiprocessing.cpu_count(),
        help="Number of FastKafka instances to run, defaults to number of CPU cores.",
    ),
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    ),
) -> None:
    try:
        asyncio.run(run_fastkafka_server(num_workers=num_workers, app=app))
    except Exception as e:
        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)

In [ ]:
# | notest

! nbdev_export

In [ ]:
result = runner.invoke(_app, ["run", "--help"])

                                                                                                                   
 Usage: root run [OPTIONS] APP                                                                                     
                                                                                                                   

Runs Fast Kafka API application

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    app      TEXT  input in the form of 'path:app', where **path** is the path to a python file and **app** is │
│                     an object of type **FastKafka**.                                                            │
│                     [default: None]                                                                             │
│                     [required]                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --num-workers        INTEGER  Number of FastKafka instances to run, defaults to number of CPU cores.            │
│                               [default: 4]                                                                      │
│ --help                        Show this message and exit.                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
with generate_app_in_tmp() as app:
    proc = await asyncio.create_subprocess_exec(
        "fastkafka", "run", app, stdout=asyncio.subprocess.PIPE
    )
    time.sleep(5)
    proc.terminate()
    await proc.wait()
    outputs, errs = await proc.communicate()

    print(outputs.decode("utf-8"))
    assert proc.returncode == 0, proc.returncode

[4174]: [INFO] main: check
[4174]: [INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[4174]: [INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[4174]: [INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[4174]: [INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[4174]: [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[4174]: [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092', 'group_id': 'tvrtko-fastkafka-kafka-1:9092_group', 'auto_offset_reset': 'earlies

In [ ]:
# | export


@_docs_app.command(
    "generate",
    help="Creates documentation for a Fast Kafka API application ",
)
def generate_docs(
    root_path: str = typer.Option(
        ".", help="root path under which documentation will be created"
    ),
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    ),
) -> None:
    try:
        application = _import_from_string(app)
        application.skip_docs = False
        application.create_docs()
    except Exception as e:
        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)


@_docs_app.command(
    "serve",
    help="Creates documentation for a Fast Kafka API application ",
)
def serve_docs(
    root_path: str = typer.Option(
        ".", help="root path under which documentation will be created"
    ),
    bind: str = typer.Option("127.0.0.1", help="Some info"),
    port: int = typer.Option(8000, help="Some info"),
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    ),
) -> None:
    try:
        application = _import_from_string(app)
        application.create_docs()
        with change_dir("asyncapi/docs/"):
            server_address = (bind, port)
            handler = SimpleHTTPRequestHandler

            # httpd = HTTPServer(server_address, handler)

            d = {"should_stop": False}

            def sigint_handler(
                signal: int, frame: Optional[FrameType], d: Dict[str, bool] = d
            ) -> None:
                d["should_stop"] = True

            signal.signal(signal.SIGINT, sigint_handler)
            signal.signal(signal.SIGTERM, sigint_handler)

            with socketserver.TCPServer(server_address, handler) as httpd:
                httpd.timeout = 0.1
                typer.secho(
                    f"Serving documentation on http://{server_address[0]}:{server_address[1]}"
                )
                while not d["should_stop"]:
                    httpd.handle_request()
                typer.secho(f"Interupting serving of documentation and cleaning up...")
    except Exception as e:
        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)

In [ ]:
# | notest

! nbdev_export

In [ ]:
result = runner.invoke(_app, ["docs", "generate", "--help"])

                                                                                                                   
 Usage: root docs generate [OPTIONS] APP                                                                           
                                                                                                                   

Creates documentation for a Fast Kafka API application

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    app      TEXT  input in the form of 'path:app', where **path** is the path to a python file and **app** is │
│                     an object of type **FastKafka**.                                                            │
│                     [default: None]                                                                             │
│                     [required]                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --root-path        TEXT  root path under which documentation will be created [default: .]                       │
│ --help                   Show this message and exit.                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
with generate_app_in_tmp() as import_str:
    result = runner.invoke(_app, ["docs", "generate", import_str])
    typer.echo(result.output)
    assert result.exit_code == 0

    result = runner.invoke(_app, ["docs", "generate", import_str])
    typer.echo(result.output)
    assert result.exit_code == 0

[INFO] main: check
[INFO] fastkafka._components.asyncapi: Old async specifications at '/tmp/tmpj2npa12c/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] fastkafka._components.asyncapi: New async specifications generated at: '/tmp/tmpj2npa12c/asyncapi/spec/asyncapi.yml'
[INFO] fastkafka._components.asyncapi: Async docs generated at 'asyncapi/docs'
[INFO] fastkafka._components.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag asyncapi/spec/asyncapi.yml @asyncapi/html-template -o asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /tmp/tmpj2npa12c/asyncapi/docs.






In [ ]:
result = runner.invoke(_app, ["docs", "serve", "--help"])

                                                                                                                   
 Usage: root docs serve [OPTIONS] APP                                                                              
                                                                                                                   

Creates documentation for a Fast Kafka API application

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    app      TEXT  input in the form of 'path:app', where **path** is the path to a python file and **app** is │
│                     an object of type **FastKafka**.                                                            │
│                     [default: None]                                                                             │
│                     [required]                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --root-path        TEXT     root path under which documentation will be created [default: .]                    │
│ --bind             TEXT     Some info [default: 127.0.0.1]                                                      │
│ --port             INTEGER  Some info [default: 8000]                                                           │
│ --help                      Show this message and exit.                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
with generate_app_in_tmp() as app:
    proc = await asyncio.create_subprocess_exec(
        "fastkafka",
        "docs",
        "serve",
        "--port=48000",
        app,
        stdout=asyncio.subprocess.PIPE,
    )
    time.sleep(30)
    proc.terminate()
    await proc.wait()
    outputs, errs = await proc.communicate()
    assert proc.returncode == 0, proc.returncode
    print(outputs.decode("utf-8"))

[INFO] main: check
[INFO] fastkafka._components.asyncapi: Old async specifications at '/tmp/tmp2ral88ln/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] fastkafka._components.asyncapi: New async specifications generated at: '/tmp/tmp2ral88ln/asyncapi/spec/asyncapi.yml'
[INFO] fastkafka._components.asyncapi: Async docs generated at 'asyncapi/docs'
[INFO] fastkafka._components.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag asyncapi/spec/asyncapi.yml @asyncapi/html-template -o asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /tmp/tmp2ral88ln/asyncapi/docs.


Serving documentation on http://127.0.0.1:48000
Interupting serving of documentation and cleaning up...

